| 项目                                                                                                                                                                           | **DDPM (随机采样)**                                                                                   | **DDIM (隐式采样)**                     |                                   |
| :--------------------------------------------------------------------------------------------------------------------------------------------------------------------------- | :------------------------------------------------------------------------------------------------ | :---------------------------------- | --------------------------------- |
| **目标**                                                                                                                                                                       | 采样 $x_{t-1} \sim p_\theta(x_{t-1}                                                                 | x_t)$                               | 构造确定性映射 $x_t \leftrightarrow x_0$ |
| **噪声预测模型**                                                                                                                                                                   | $\epsilon_\theta(x_t, t)$                                                                         | 相同                                  |                                   |
| **预测 $x_0$**                                                                                                                                                                 | $\hat{x}_0 = \frac{x_t - \sqrt{1-\bar{\alpha}*t},\epsilon*\theta(x_t, t)}{\sqrt{\bar{\alpha}_t}}$ | 相同                                  |                                   |
| **反推公式**                                                                                                                                                                     |                                                                                                   |                                     |                                   |
| $\displaystyle x_{t-1} = \frac{1}{\sqrt{\alpha_t}}\Big(x_t - \frac{1-\alpha_t}{\sqrt{1-\bar{\alpha}*t}}\epsilon*\theta(x_t, t)\Big) + \sigma_t z,\ \ z\sim\mathcal{N}(0, I)$ |                                                                                                   |                                     |                                   |
| $\displaystyle x_{t-1} = \sqrt{\bar{\alpha}*{t-1}}\hat{x}*0 + \sqrt{1-\bar{\alpha}*{t-1}}\Big( \sqrt{1-\eta_t^2},\epsilon*\theta(x_t, t) + \eta_t z\Big)$                    |                                                                                                   |                                     |                                   |
| **随机项**                                                                                                                                                                      | $\sigma_t z$，其中 $\sigma_t^2 = \tilde{\beta}_t$（posterior variance）                                | $\eta_t$ 控制随机性，常设 $\eta_t=0$ 得确定性采样 |                                   |
| **马尔可夫性**                                                                                                                                                                    | 马尔可夫链，每步重新采样噪声                                                                                    | 非马尔可夫链，显式确定轨迹                       |                                   |
| **特殊情况**                                                                                                                                                                     | $\sigma_t > 0$ → 随机                                                                               | $\eta_t = 0$ → 确定性（Implicit）        |                                   |
| **计算路径**                                                                                                                                                                     | $$x_t \to x_{t-1} \to \cdots \to x_0$$                                                            | $$x_T \to x_t \leftrightarrow x_0$$ |                                   |
| **采样步数**                                                                                                                                                                     | 通常 1000+                                                                                          | 10~100 步即可                          |                                   |


## 🧩 DDPM → DDIM 推导笔记 (for ipynb Markdown)

### (A) DDPM 反向生成过程

在前向扩散（加噪）过程中：

$$
q(\mathbf{x}_t \mid \mathbf{x}_0)
= \mathcal{N}\big(
\sqrt{\bar{\alpha}_t}\,\mathbf{x}_0,\;
(1-\bar{\alpha}_t)\,\mathbf{I}
\big)
$$

其中 $\alpha_t = 1 - \beta_t$，$\bar{\alpha}_t = \prod_{\tau=1}^{t} \alpha_\tau$。


反向生成过程为：


$$
p_\theta(\mathbf{x}_{t-1} \mid \mathbf{x}_t)
= \mathcal{N}\big(
\boldsymbol{\mu}_\theta(\mathbf{x}_t,t),
\sigma_t^2 \mathbf{I}
\big)
$$

其中：

$$
\boldsymbol{\mu}_\theta(\mathbf{x}_t,t)
= \frac{1}{\sqrt{\alpha_t}}
\left(
\mathbf{x}_t- \frac{1-\alpha_t}{\sqrt{1-\bar{\alpha}_t}}
  \varepsilon_\theta(\mathbf{x}_t,t)
  \right),
  \quad
  \sigma_t^2 = \beta_t.
  $$

每一步的更新为：

$$
\mathbf{x}_{t-1}
= \boldsymbol{\mu}_\theta(\mathbf{x}_t,t) + \sigma_t \mathbf{z},
  \quad \mathbf{z}\sim\mathcal{N}(\mathbf{0},\mathbf{I})
  $$

这说明 DDPM 是一个**随机马尔可夫链**，反向采样依赖噪声 $\mathbf{z}$。


### (B) Tweedie 视角：重建 $\hat{\mathbf{x}}_0$

在前向过程中：

$$
\mathbf{x}_t
= \sqrt{\bar{\alpha}_t}\,\mathbf{x}_0
+ \sqrt{1-\bar{\alpha}_t}\,\boldsymbol{\epsilon},
\quad
\boldsymbol{\epsilon} \sim \mathcal{N}(\mathbf{0}, \mathbf{I})
$$

于是我们可以通过预测噪声 $\varepsilon_\theta(\mathbf{x}_t, t)$ 来估计干净信号：

$$
\hat{\mathbf{x}}_0(\mathbf{x}_t, t)
= \frac{1}{\sqrt{\bar{\alpha}_t}}
\left(
\mathbf{x}_t
- \sqrt{1-\bar{\alpha}_t}\,
\varepsilon_\theta(\mathbf{x}_t, t)
\right)
\tag{1}
$$

即 Tweedie 公式的扩散版：

$$
\hat{\mathbf{x}}_0 \approx \mathbb{E}[\mathbf{x}_0 \mid \mathbf{x}_t]
$$


---

### (C) DDPM 的通用反向形式

Song 等人（2020）将 DDPM 的反向采样重写为：

$$
\mathbf{x}_{t-1}
=
\sqrt{\bar{\alpha}_{t-1}}\,
\hat{\mathbf{x}}_0(\mathbf{x}_t, t)
+
\sqrt{1-\bar{\alpha}_{t-1}-\sigma_t^2}\,
\varepsilon_\theta(\mathbf{x}_t, t)
+
\sigma_t\,\mathbf{z},
\quad
\mathbf{z} \sim \mathcal{N}(\mathbf{0}, \mathbf{I})
\tag{2}
$$

其中 $\sigma_t$ 决定了反向过程的随机性。DDPM 中 $\sigma_t^2 = \beta_t$，因此该过程是随机的。


---

### (D) DDIM 的关键假设与推导

DDIM 的出发点是：

> 只要边缘分布 $q(\mathbf{x}_t \mid \mathbf{x}_0)$ 相同，反向过程不必是马尔可夫随机的。

因此定义新的“隐式反向过程”：

$$
q_\sigma(\mathbf{x}_{t-1} \mid \mathbf{x}_t, \mathbf{x}_0)
= \mathcal{N}\!\left(
\mathbf{x}_{t-1};
\sqrt{\bar{\alpha}_{t-1}}\,\mathbf{x}_0
+ \sqrt{1-\bar{\alpha}_{t-1}-\sigma_t^2}\,
\varepsilon_\theta(\mathbf{x}_t, t),
\sigma_t^2 \mathbf{I}
\right)
$$

若令 $\sigma_t = 0$，即不再添加随机噪声，则更新公式化为：

$$
\mathbf{x}_{t-1}
=
\sqrt{\bar{\alpha}_{t-1}}\,
\hat{\mathbf{x}}_0(\mathbf{x}_t, t)
+
\sqrt{1-\bar{\alpha}_{t-1}}\,
\varepsilon_\theta(\mathbf{x}_t, t)
\tag{3}
$$


### (E) 代入 $\hat{\mathbf{x}}_0$ 表达式

将式 (1) 代入 (3)，得到仅依赖 $(\mathbf{x}_t, \varepsilon_\theta)$ 的更新式：

$$
\begin{aligned}
\mathbf{x}_{t-1}
&=
\sqrt{\bar{\alpha}_{t-1}}\,
\frac{1}{\sqrt{\bar{\alpha}_t}}
\left(
\mathbf{x}_t
-
\sqrt{1-\bar{\alpha}_t}\,
\varepsilon_\theta(\mathbf{x}_t, t)
\right)
+
\sqrt{1-\bar{\alpha}_{t-1}}\,
\varepsilon_\theta(\mathbf{x}_t, t)
\\[4pt]
&=
\sqrt{\frac{\bar{\alpha}_{t-1}}{\bar{\alpha}_t}}\,
\mathbf{x}_t
+
\left(
\sqrt{1-\bar{\alpha}_{t-1}}
-
\sqrt{\frac{\bar{\alpha}_{t-1}}{\bar{\alpha}_t}}\,
\sqrt{1-\bar{\alpha}_t}
\right)
\varepsilon_\theta(\mathbf{x}_t, t)
\end{aligned}
\tag{4}
$$

这就是 **DDIM 核心采样公式**。

---

### (F) 模型关系总结

| 模型 | 方程类型 | 随机性 | 表达式 |
|------|-----------|-----------|-----------|
| **DDPM** | 随机微分方程 (SDE) | 有噪声项 $\sigma_t \mathbf{z}$ | $$\mathbf{x}_{t-1} = \mu_\theta(\mathbf{x}_t, t) + \sigma_t \mathbf{z}$$ |
| **DDIM** | 常微分方程 (ODE) | 确定性路径 ($\sigma_t = 0$) | $$\mathbf{x}_{t-1} = \sqrt{\bar{\alpha}_{t-1}}\,\hat{\mathbf{x}}_0 + \sqrt{1-\bar{\alpha}_{t-1}}\,\varepsilon_\theta(\mathbf{x}_t, t)$$ |

因此 DDIM 可以看作是 **在保持相同边缘分布的前提下，将 DDPM 的随机 SDE 路径改写成确定性 ODE 轨迹**，从而减少采样步数、提高速度。
